In [ ]:
from data_loader import MyPro,convert_examples_to_features,create_batch_iter
# from bert_ner import Bert_Softmax
# from bert_ner import Bert_Softmax
import torch
from transformers import BertConfig,AdamW

import torch.nn as nn

import torch.optim as optim

from Evalutor import evaluate

In [ ]:
from transformers import BertPreTrainedModel, BertModel

In [ ]:
import json

### 创建数据entitys

In [ ]:
with open('multi_data/entitys.json','r',encoding='utf-8') as f:
    entitys = json.load(f)
f.close()
print(len(entitys))

In [ ]:
tag2id = {}
tag2id['O'] = 0
tag2id['traffic_in'] = 1
tag2id['sell_drugs_to'] = 2
tag2id['provide_shelter_for'] = 3
tag2id['posess'] = 4
id2tag = ['O','traffic_in','sell_drugs_to','provide_shelter_for','posess']

### 创建model

In [ ]:
class JS_Model(BertPreTrainedModel):
    def __init__(self,
                 config,
                 num_tag):
        super(JS_Model, self).__init__(config)
        self.bert = BertModel.from_pretrained('../bert-base-chinese/', config=config) #.cuda()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_tag)

    def forward(self,
                input_ids,
                token_type_ids,
                attention_mask,mode):

        bert_encode = self.bert(input_ids, attention_mask, token_type_ids)  ## 
#         bert_encode = torch.tensor(bert_encode).cuda()
        if mode =="ner":
            bert_encode0 = bert_encode[0].cuda()
            output = self.classifier(bert_encode0)
            output = F.log_softmax(output, dim=2)
        elif mode == "re":
            bert_encode1 = bert_encode[1].cuda()
            output = self.classifier(bert_encode1)
            
        return output

In [ ]:
'''
model --->Shared Parameters
'''


config = BertConfig.from_json_file('../bert-base-chinese/bert_config.json')
model = JS_Model(config,5).cuda()

In [ ]:
### 冻结部分参数

unfreeze_layers = ['layer.11','bert.pooler','classifier.']
 
for name, param in model.named_parameters():
    print(name,param.size())

print("*"*30)
print('\n')

for name ,param in model.named_parameters():
    param.requires_grad = False
    for ele in unfreeze_layers:
        if ele in name:
            param.requires_grad = True
            break
#验证一下
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name,param.size())

In [ ]:
#过滤掉requires_grad = False的参数
'''
  #### 优化器
'''
lr = 0.0001
optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr,weight_decay = 0.1)

### 涉及到NER的参数/变量

In [ ]:
import torch.nn.functional as F

In [ ]:
from SP_data_loader import create_batch_iter

In [ ]:
### dataset
label_list = ['O', 'B-Nh', 'I-Nh', 'B-NDR', 'I-NDR']
train_iter = create_batch_iter("train",'multi_data/multi_train_er.txt',12,label_list)

In [ ]:
# for step, batch in enumerate(train_iter):
# #     count += 1
# #     input_ids, input_mask, segment_ids, label_ids,others = batch   ##(batch_size,512)
# #     break

# others

In [ ]:
weight = [10.0] * len(label_list)   #生成一维
weight[0] = 1
weight = torch.tensor(weight).cuda()
criterion = nn.NLLLoss(weight,size_average=False)      #定义的损失函数，softmax以后求损失函数

### 涉及到RE的参数/变量/函数

In [ ]:
import json

In [ ]:
from collections import Counter

In [ ]:
import random

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer('../bert-base-chinese/vocab.txt')

In [ ]:
with open('multi_data/multi_train_er.txt','r',encoding='utf-8') as f:
    flines = f.readlines()

In [ ]:
ent_tag = [json.loads(line) for line in flines]

In [ ]:
def create_re_lines(out,label_ids,input_mask):
    out = torch.argmax(out,dim= 2)
    lines = []
    for i in range(len(out)):
        s_lines = []
        label_true = [label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ]
        y_pre = [out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ]
        y_pre = torch.tensor(y_pre)
        valid_pos = torch.nonzero(y_pre).squeeze(1)

        entitys = []
#         entitys2 = []
        flag = valid_pos[0]
        start = flag
        end=start
        for vp in valid_pos[1:]:
            if vp -1 == flag:
                end = vp
            elif start==end:
                continue
            else:
                entity = tokenizer.decode(input_ids[i][start:end+1])
                entity = entity.replace(' ','')
                if entity == '':
                    continue
                entitys.append(entity)
                start = vp

            flag = vp
        entity = tokenizer.decode(input_ids[i][start:valid_pos[-1]+1])
        entity = entity.replace(' ','')
        entitys.append(entity)   ### 最后一个实体
#         print(entitys)
        context = tokenizer.decode(input_ids[i][:len(y_pre)][1:-1])
        context = context.replace(' ','')
#         context = context[1:-1]


        entitys = list(set(entitys))   ### 去重，无所谓顺序，因为两两实体都会进行组合
#         e1_e2 = []
        e_len = len(entitys)
        for i in range(e_len):
            e1 = entitys[i]
            for j in range(e_len):
                if i == j:
                    continue
                e2 = entitys[j]
                line = e1+'，'+e2 #+'。'+context
                s_lines.append(line)
        lines.append(s_lines)
                
    return lines

In [ ]:
# input_ids[0].shape

# x = torch.tensor([0]).cuda()

# len(segment_ids[0])

In [ ]:
# lines = create_re_lines(out,label_ids,input_mask)

# print(lines)

In [ ]:
def get_true_trip(others):
    valid_tris = []
    for other in others:
        valid_tri = []
        for eel in other:

            if eel[2] == 0:
                break
            valid_tri.append((entitys[eel[0]],entitys[eel[1]],eel[2].item()))
        valid_tris.append(valid_tri)
        
    return valid_tris

In [ ]:
# true_tris = get_true_trip(others)

In [ ]:
### 比较二给定的双实体 获得在原句中具有关系
## bi --> (pre_e1,pre_e2)  
## tris --> [(e1,e2,rela),...]
def get_rela(bi,tris):
    
    bi = bi.split('，')
#     print(bi)
#     print(tris)
    for tri in tris:
#         print(tri)
#         tri = tri.split(',')
        if bi[0] == tri[0] and bi[1] == tri[1]:
            return tri[2]
        
    return 0
    

In [ ]:
## segment_ids 不需要更新 因为全为0
def handle_ids_mask_seg(ids,mask,in_id): ## in_id [CLS] 习某某，何某某。
    length = len(in_id)
    
    in_id = torch.tensor(in_id).cuda()
    new_ids = torch.cat((in_id,ids[1:]))[:512]
    
    _mask = torch.tensor([1]*length).cuda()
    new_mask = torch.cat((_mask,mask[1:]))[:512]
    
#     _seg = torch.tensor([0]*length).cuda()
#     new_seg = torch.cat((_seg))   seg不用变  因为全0
    return new_ids,new_mask
#     pass

In [ ]:
def get_new_id_seg_mask_label(lines,true_tris,input_ids,input_mask,segment_ids):
    assert len(lines) == len(true_tris)

    length = len(lines)

    new_ids = []
    new_masks = []
    new_seg_ids = []
    new_labels = []
    for i in range(length):
#         print(float(i))
        input_id = input_ids[i]
        input_m = input_mask[i]
        segment_id = segment_ids[i]
        line = lines[i]
        true_tri = true_tris[i]
    #     print(true_tri)
        for l in line:
    #         print(true_tri)
            label = get_rela(l,true_tri)
    #         if label != 0:
#             print(label)
            tokens = ["[CLS]"] + list(l+'。')
            in_id = tokenizer.convert_tokens_to_ids(tokens)

    #             print(l)
    #             print(tokens)
    #             print(in_id)
            assert len(tokens) == len(in_id)
            new_id,new_mask = handle_ids_mask_seg(input_id,input_m,in_id)

            assert Counter(new_id.tolist())[0] == Counter(new_mask.tolist())[0]

            new_ids.append(new_id.tolist())
            new_masks.append(new_mask.tolist())
            new_seg_ids.append(segment_id.tolist())
            new_labels.append(label)

    re_len = len(new_ids)
    index = list(range(re_len))
    random.shuffle(index)

    new_idss = [new_ids[i] for i in index]
    new_maskss = [new_masks[i] for i in index]
    new_seg_idss = [new_seg_ids[i] for i in index]
    new_labelss = [new_labels[i] for i in index]

    new_ids = torch.tensor(new_idss).cuda()
    new_masks = torch.tensor(new_maskss).cuda()
    new_seg_ids = torch.tensor(new_seg_idss).cuda()
    new_labels = torch.tensor(new_labelss).cuda()
    
    return new_ids,new_masks,new_seg_ids,new_labels

In [ ]:
# assert len(lines) == len(true_tris)

# length = len(lines)

# new_ids = []
# new_masks = []
# new_seg_ids = []
# new_labels = []
# for i in range(length):
#     print(float(i))
#     input_id = input_ids[i]
#     input_m = input_mask[i]
#     segment_id = segment_ids[i]
#     line = lines[i]
#     true_tri = true_tris[i]
# #     print(true_tri)
#     for l in line:
# #         print(true_tri)
#         label = get_rela(l,true_tri)
# #         if label != 0:
#         print(label)
#         tokens = ["[CLS]"] + list(l+'。')
#         in_id = tokenizer.convert_tokens_to_ids(tokens)

# #             print(l)
# #             print(tokens)
# #             print(in_id)
#         assert len(tokens) == len(in_id)
#         new_id,new_mask = handle_ids_mask_seg(input_id,input_m,in_id)

#         assert Counter(new_id.tolist())[0] == Counter(new_mask.tolist())[0]
        
#         new_ids.append(new_id.tolist())
#         new_masks.append(new_mask.tolist())
#         new_seg_ids.append(segment_id.tolist())
#         new_labels.append(label)

# #             break
# #         print(label)
        

# import random

# re_len = len(new_ids)
# index = list(range(re_len))
# random.shuffle(index)

# new_idss = [new_ids[i] for i in index]
# new_maskss = [new_masks[i] for i in index]
# new_seg_idss = [new_seg_ids[i] for i in index]
# new_labelss = [new_labels[i] for i in index]

# new_ids = torch.tensor(new_idss).cuda()
# new_masks = torch.tensor(new_maskss).cuda()
# new_seg_ids = torch.tensor(new_seg_idss).cuda()
# new_labels = torch.tensor(new_labelss).cuda()

# # out = model(input_ids,segment_ids,input_mask,"ner")  #[batch_size,seq_len+2(512)]
# re_out = model(new_ids,new_seg_ids,new_masks,'re')

# loss1 = F.cross_entropy(re_out, new_labels)

# print(loss1)

In [ ]:
# from collections import Counter

# Counter(new_id.tolist())[0]

# Counter(new_mask.tolist())[0]

### 训练

In [ ]:
model.train()
for e in range(1):
    count = 0
    if (e+1)%2 == 0:
        lr = lr/2
        optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr,weight_decay = 0.1)
    
    for step, batch in enumerate(train_iter):
        count += 1
        
        input_ids, input_mask, segment_ids, label_ids,others = batch   ##(batch_size,512)
        
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        label_ids = label_ids.cuda()
        out = model(input_ids,segment_ids,input_mask,"ner")  #[batch_size,seq_len+2(512)]
        
        '''
        计算loss
        '''

        out1 = out.view(-1, out.size(2))
        res1 = label_ids[0]
        for i in range(1,len(label_ids)):
            res1 = torch.cat((res1,label_ids[i]))
        loss = criterion(out1,res1)
        loss = loss/len(out)
        print(loss)
        if loss <10.0:
            
#             lines = create_re_lines(out,label_ids,input_mask)
            break
    
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(str(e)+'：结束')

In [ ]:
model.train()
for e in range(1):
    count = 0
    if (e+1)%2 == 0:
        lr = lr/2
        optimizer = AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=lr,weight_decay = 0.1)
    
    for step, batch in enumerate(train_iter):
        count += 1
        
        input_ids, input_mask, segment_ids, label_ids,others = batch   ##(batch_size,512)
        
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()
        segment_ids = segment_ids.cuda()
        label_ids = label_ids.cuda()
        out = model(input_ids,segment_ids,input_mask,"ner")  #[batch_size,seq_len+2(512)]
        
        '''
        计算loss
        '''

        out1 = out.view(-1, out.size(2))
        res1 = label_ids[0]
        for i in range(1,len(label_ids)):
            res1 = torch.cat((res1,label_ids[i]))
        loss1 = criterion(out1,res1)
        loss1 = loss1/len(out)
        
        if loss1 <5.0:
            print("re loss")
            lines = create_re_lines(out,label_ids,input_mask)
            true_tris = get_true_trip(others)

            new_ids,new_masks,new_seg_ids,new_label = get_new_id_seg_mask_label(lines,true_tris,input_ids,input_mask,segment_ids)
            re_out = model(new_ids,new_seg_ids,new_masks,'re')
            loss2 = F.cross_entropy(re_out, new_labels)
        else:
            loss2 = loss1
        
        loss = 0.5*(loss1+loss2)
        
        print(loss)
#         print(loss)
#         if loss <10.0:
            
# #             lines = create_re_lines(out,label_ids,input_mask)
#             break
        
        
        '''
        产生re数据
        '''
        
#         lines = create_re_lines(out,label_ids,input_mask)
#         out = torch.argmax(out,dim= 2)
#         lines = []
#         for i in range(len(out)):
#             label_true = [label_ids[i][j].item() for j in range(512) if input_mask[i][j] == 1 ]
#             y_pre = [out[i][j].tolist() for j in range(512) if input_mask[i][j] == 1 ]
#             y_pre = torch.tensor(y_pre)
#             valid_pos = torch.nonzero(y_pre).squeeze(1)

#             entitys = []
#     #         entitys2 = []
#             flag = valid_pos[0]
#             start = flag
#             for vp in valid_pos[1:]:
#                 if vp -1 == flag:
#                     end = vp
#                 else:
#                     entity = tokenizer.decode(input_ids[i][start:end+1])
#                     entity = entity.replace(' ','')
#                     entitys.append(entity)
#                     start = vp

#                 flag = vp
#             entity = tokenizer.decode(input_ids[i][start:valid_pos[-1]+1])
#             entity = entity.replace(' ','')
#             entitys.append(entity)   ### 最后一个实体
#     #         print(entitys)
#             context = tokenizer.decode(input_ids[i][:len(y_pre)][1:-1])
#             context = context.replace(' ','')
#     #         context = context[1:-1]


#             entitys = list(set(entitys))   ### 去重，无所谓顺序，因为两两实体都会进行组合
#     #         e1_e2 = []
#             e_len = len(entitys)
#             for i in range(e_len):
#                 e1 = entitys[i]
#                 for j in range(e_len):
#                     if i == j:
#                         continue
#                     e2 = entitys[j]
#                     line = e1+'，'+e2+'。'+context
#                     lines.append(line)
#         break
        
#         if count %5 == 0:
#             print('loss: ',loss)
#             out2 = torch.argmax(out,dim=2)
#             batch_precision,batch_recall,batch_f1 = evaluate(out2,label_ids,input_mask)
#             print('batch_precision:%.4f  batch_recall:%.4f  batch_f1: %.4f' %(batch_precision,batch_recall,batch_f1)) 
            
#             bor_precision, bor_recall,bor_f1 = evaluate_bor(out2,label_ids,input_mask)
#             print('bor_precision:%.4f  bor_recall:%.4f  bor_f1: %.4f' %(bor_precision,bor_recall,bor_f1)) 
#         print(lines)
#         break
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(str(e)+'：结束')
        
        

In [ ]:
new_ids.shape

In [ ]:
valid_tris = []
for other in others:
    valid_tri = []
    for eel in other:
        
        if eel[2] == 0:
            break
        valid_tri.append((entitys[eel[0]],entitys[eel[1]],id2tag[eel[2]]))
    valid_tris.append(valid_tri)

In [ ]:
valid_tris

In [ ]:
others

In [ ]:
a = others[0][0]

entitys[a[1]]
# entitys[a[0]]

In [ ]:
entitys[a[0]]

In [ ]:
lines = create_re_lines(out,label_ids,input_mask)

In [ ]:
lines[2]

In [ ]:
a= torch.tensor([1,2])

In [ ]:
b = torch.tensor([3])

In [ ]:
torch.cat((a,b))

In [ ]:
a.shape

In [ ]:
a = a.cuda()

In [ ]:
c= input_ids[0]

In [ ]:
c.shape

In [ ]:
torch.cat((a,input_ids[0]))